In [ ]:
# Import the libraries we will use
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#from sklearn.metrics import mean_squared_error
import plotly.offline as pl
import plotly.graph_objs as go
pl.init_notebook_mode(connected=True)
#from datetime import datetime

In [ ]:
#exchange = ('bitstamp_btc_usd','cex_btc_usd','exmo_btc_usd','bitfinex_btc_usd','kraken_btc_usd', 'gdax_btc_usd')
exchange = ('kraken_btc_usd', 'bitstamp_btc_usd')
time = ('10s','20s','30s')
#alpha_all = (1.5,2,2.5,3)
alpha_all = (2,3)
#n = 0 # Exchange number
#S = 2 # SPREAD > S
#time_period = '10s' # '60s' '30min' -- prediction time
#alpha = 3 # K > alpha

In [ ]:
for S in range(1,4):
    for alpha in alpha_all:
        for time_period in time:
            for n in range(len(exchange)):
                try:
                    print(exchange[n])
                    print(time_period)
                    print(S,'$')
                    print('alpha = ',alpha)
                    # Read ask-bid price from the file
                    series_exch = pd.read_csv(exchange[n]+'.csv', index_col=0)
                    series_exch.index = pd.to_datetime(series_exch.index, unit='ms') # Time converting and +3 hour
                    #series_exch.tail()
                    series_period = series_exch.resample(time_period).mean()
                    series_period['spread'] = series_period['ask'] - series_period['bid']
                    #series_period.tail()
                    N = len(series_period.index)
                    BID = series_period['bid']
                    ASK = series_period['ask']
                    SPREAD = series_period['spread']
                    K = np.ones(N-2)
                    D = np.zeros(N-2)
                    F = np.ones(N-2)
                    SPREADbefore = np.zeros(N-2)
                    SPREADnow = np.zeros(N-2)
                    BIDbefore = np.zeros(N-2)
                    ASKbefore = np.zeros(N-2)
                    for t in range(1,N-2):
                        if ASK[t]-ASK[t-1]>0 and BID[t]-BID[t-1]>-0.25*SPREAD[t-1]:
                            K[t] = SPREAD[t]/SPREAD[t-1]
                            D[t] = BID[t+1]-BID[t]
                            F[t] = BID[t+1]/BID[t]
                            SPREADbefore[t] = SPREAD[t-1]
                            SPREADnow[t] = SPREAD[t]
                    # Creating dataframe and log-transform y
                    #data = {'ask': ASK[1:N-1], 'ask_before': ASKbefore, 'bid': BID[1:N-1], 'bid_before': BIDbefore, 'SPREAD': SPREAD[1:N-1], 'SPREAD_before': SPREADbefore, 'K': K, 'D': D, 'F': F}
                    #data = {'bid': BID[1:N-1], 'bid_before': BIDbefore, 'SPREAD': SPREAD[1:N-1], 'SPREAD_before': SPREADbefore, 'K': K, 'D': D, 'F': F}
                    data = {'SPREAD': SPREADnow, 'SPREAD_before': SPREADbefore, 'K': K, 'D': D, 'F': F}
                    df_data = pd.DataFrame(data=data)
                    df_data['K_check'] = df_data['SPREAD']/df_data['SPREAD_before']
                    #print(df_data[['SPREAD_before', 'SPREAD', 'K', 'K_check']])
                    df_data = df_data[df_data['K']>alpha].copy() # K>alpha
                    df_data = df_data[df_data['SPREAD_before']>S].copy()
                    #df_data['K_check'] = df_data['SPREAD']/df_data['SPREAD_before']
                    #print(df_data[['SPREAD_before', 'SPREAD', 'K', 'K_check']])
                    #df_data_plus = df_data[df_data['D']>0].copy()
                    print('Количество', len(df_data))
                    print('Отношение', len(df_data_plus)/len(df_data))
                    K = df_data['K'].values
                    D = df_data['D'].values
                    F = df_data['F'].values
                    print('Пирсон', pearsonr(K, D))
                    #pearsonr(K, F)
                    bid_ind = go.Scatter(
                                x = K,
                                y = D,
                                name = 'BID',
                                mode = 'markers'         
                    )
                    data = [bid_ind]
                    layout = dict(title = 'BTC bid price',
                    xaxis = dict(title = 'K'),
                    yaxis = dict(title = 'D'),
                    #paper_bgcolor='rgb(255,255,255)',
                    #plot_bgcolor='rgb(229,229,229)',
                    )    
                    fig = dict(data=data, layout=layout)
                    pl.iplot(fig, filename='styled-line')
                except:
                    pass